# Download data from warehouse -> add it to our private GCS bucket as a parquet -> clean it -> add it to the public GCS bucket

In [1]:
from datetime import datetime
from pathlib import Path
from typing import Literal

import _sql_query
import geopandas as gpd
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas

import pandas as pd
from functools import cache

In [2]:
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils, geo_utils

In [3]:
import publish_public_data
from calitp_data_analysis import geography_utils
from shared_utils import gtfs_utils_v2, portfolio_utils, publish_utils, geo_utils
from update_vars import GTFS_DATA_DICT

In [4]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [5]:
PUBLIC_GCS = GTFS_DATA_DICT.gcs_paths.PUBLIC_GCS

In [6]:
@cache
def gcs_geopandas():
    return GCSGeoPandas()

In [7]:
date_str = datetime.today().strftime("%Y_%m")

In [8]:
date_str

'2025_12'

## 1) schedule_rt_route_direction_summary
* Add Caltrans District to organize 

In [9]:
def download_schedule_rt_route_direction_summary(download_date: str) -> pd.DataFrame:
    df = _sql_query.download_with_pandas_gbq(
        project="cal-itp-data-infra-staging",
        filename="tiffany_mart_gtfs_rollup.fct_monthly_schedule_rt_route_direction_summary",
    )

    # Save to our private GCS
    df.to_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/schedule_rt_route_direction_summary_{download_date}.parquet"
    )

    return df

In [25]:
og_download_schedule_rt_route_direction_summary = download_schedule_rt_route_direction_summary(date_str)


        SELECT 
            *
        FROM `cal-itp-data-infra-staging`.`tiffany_mart_gtfs_rollup`.`fct_monthly_schedule_rt_route_direction_summary`
        WHERE month_first_day >=  DATE('2025-01-01')
    
Downloading: 100%|██████████|
download time: 0:01:14.756786


In [10]:
def prep_schedule_rt_route_direction_summary(download_date: str) -> pd.DataFrame:
    df = download_schedule_rt_route_direction_summary(download_date)

    df2 = df[
        [
            "month_first_day",
            "name",
            "route_name",
            "direction_id",
            "frequency_all_day",
            "frequency_offpeak",
            "frequency_peak",
            "daily_service_hours",
            "daily_trips_peak",
            "daily_trips_offpeak",
            "daily_trips_all_day",
            "day_type",
        ]
    ]
    # Temporarily add analysis_name
    df2 = df2.pipe(portfolio_utils.standardize_portfolio_organization_names)
    df2 = df2.drop_duplicates()

    # Clean columns
    df2.columns = df2.columns.str.replace("_", " ").str.title()
    df2["Month First Day"] = pd.to_datetime(df2["Month First Day"]).dt.strftime("%B %Y")
    df2 = df2.rename(
        columns={
            "Direction Id": "Direction",
            "Month First Day": "Date",
            "Route Name": "Route",
        }
    )
    # Add some new columns
    df2["Daily Service Minutes"] = df2["Daily Service Hours"] * 60
    df2["Average Scheduled Minutes"] = (
        df2["Daily Service Minutes"] / df2["Daily Trips All Day"]
    )
    df2["Average Scheduled Minutes"].describe()
    df2["Headway All Day"] = 60 / df2["Frequency All Day"]
    df2["Headway Peak"] = 60 / df2["Frequency Peak"]
    df2["Headway Offpeak"] = 60 / df2["Frequency Offpeak"]

    # Save
    df2.to_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/schedule_rt_route_direction_summary_{download_date}.parquet"
    )
    return df2

In [11]:
schedule_rt_route_direction_summary = prep_schedule_rt_route_direction_summary(date_str)


        SELECT 
            *
        FROM `cal-itp-data-infra-staging`.`tiffany_mart_gtfs_rollup`.`fct_monthly_schedule_rt_route_direction_summary`
        WHERE month_first_day >=  DATE('2025-01-01')
    


/opt/conda/lib/python3.11/site-packages/pandas_gbq/gbq.py:38: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa


Downloading: 100%|██████████|
download time: 0:01:22.971806


## 2) fct_monthly_operator_summary
* Add Caltrans District + Legislative District

In [12]:
def download_operator_summary(download_date: str) -> pd.DataFrame:
    df = _sql_query.download_with_pandas_gbq(
        project="cal-itp-data-infra-staging",
        filename="tiffany_mart_gtfs_rollup.fct_monthly_operator_summary",
    )

    # Save to our private GCS
    df.to_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/fct_monthly_operator_summary_{download_date}.parquet"
    )
    return df

In [13]:
def clean_operator_summary(download_date: str) -> pd.DataFrame:
    df = download_operator_summary(download_date)

    df2 = df[
        [
            "month_first_day",
            "analysis_name",
            "vp_name",
            "tu_name",
            "n_trips",
            "day_type",
            "daily_trips",
            "ttl_service_hours",
            "n_routes",
            "n_days",
            "n_shapes",
            "n_stops",
            "vp_messages_per_minute",
            "n_vp_trips",
            "daily_vp_trips",
            "pct_vp_trips",
            "n_vp_routes",
            "pct_vp_service_hours",
            "tu_messages_per_minute",
            "n_tu_trips",
            "daily_tu_trips",
            "pct_tu_trips",
            "n_tu_routes",
            "pct_tu_service_hours",
        ]
    ]
    df2.columns = df2.columns.str.replace("_", " ").str.title()

    df2 = df2.rename(
        columns={
            "Month First Day": "Date",
        }
    )
    df2.columns = df2.columns.str.replace("Vp", "VP").str.replace("Tu", "TU")

    # Create a couple of new columns
    df2["Percent of Trips with Trip Updates"] = (
        df2["N TU Trips"] / df2["N Trips"]
    ) * 100

    df2["Percent of Trips with Vehicle Positions"] = (
        df2["N VP Trips"] / df2["N Trips"]
    ) * 100

    """
    df2["Percent of Trips with Vehicle Positions"] = df2[
        "Percent of Trips with Vehicle Positions"
    ].clip(upper=100.0)
    df2["Percent of Trips with Trip Updates"] = df2[
        "Percent of Trips with Trip Updates"
    ].clip(upper=100.0)
    """
    # Save
    df2.to_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/fct_monthly_operator_summary_{download_date}.parquet"
    )
    return df2

In [14]:
monthly_operator_summary_clean = clean_operator_summary(date_str)


        SELECT 
            *
        FROM `cal-itp-data-infra-staging`.`tiffany_mart_gtfs_rollup`.`fct_monthly_operator_summary`
        WHERE month_first_day >=  DATE('2025-01-01')
    
Downloading: 100%|██████████|
download time: 0:00:02.231177


## 3) fct_monthly_routes 
* Add Caltrans District + Legislative District

In [15]:
def download_fct_monthly_routes(download_date: str) -> pd.DataFrame:
    df = _sql_query.download_with_pandas_gbq(
        project="cal-itp-data-infra-staging",
        filename="tiffany_mart_gtfs_rollup.fct_monthly_routes",
    )

    # Save to our private GCS
    df.to_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/fct_monthly_routes_{download_date}.parquet"
    )
    return df

In [16]:
# fct_monthly_routes = download_fct_monthly_routes()

In [17]:
def clean_fct_monthly_routes(download_date: str) -> pd.DataFrame:
    df = download_fct_monthly_routes(download_date)

    # Add analysis name
    df = df.pipe(portfolio_utils.standardize_portfolio_organization_names)

    # Keep only the most recent route geography
    df2 = df.sort_values(
        by=["month_first_day", "analysis_name", "route_name"],
        ascending=[False, True, True],
    ).drop_duplicates(subset=["analysis_name", "route_name"])

    df2 = df.drop(
        columns=[
            "shape_id",
            "shape_array_key",
            "n_trips",
            "direction_id",
        ]
    )

    # Add route_type, route_color, and route_typology
    route_info_df = pd.read_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/schedule_rt_route_direction_summary_{download_date}.parquet"
    )[
        [
            "name",
            "month_first_day",
            "route_name",
            # "route_type",
            "route_color",
            "route_typology",
        ]
    ]

    m1 = pd.merge(
        df2,
        route_info_df,
        on=[
            "name",
            "month_first_day",
            "route_name",
        ],
        how="left",
    ).drop(columns = ["month_first_day"])

    
    # Convert the geometry to line
    m1 = geo_utils.convert_to_gdf(m1, "pt_array", "line")
    
    m1 = gcs_geopandas().geo_data_frame_to_parquet(
        m1,
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/fct_monthly_routes_{download_date}.parquet"
    )
    return m1

In [18]:
# fct_operator_hourly_summary = clean_fct_monthly_routes(date_str)

## 4) fct_operator_hourly_summary

In [19]:
def download_fct_operator_hourly_summary(download_date: str) -> pd.DataFrame:
    df = _sql_query.download_with_pandas_gbq(
        project="cal-itp-data-infra-staging",
        filename="tiffany_mart_gtfs_rollup.fct_operator_hourly_summary",
    )

    # Save to our private GCS
    df.to_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}raw/fct_operator_hourly_summary_{download_date}.parquet"
    )
    return df

In [20]:
# fct_operator_hourly_summary = download_fct_operator_hourly_summary()

In [21]:
def clean_fct_operator_hourly_summary(download_date: str) -> pd.DataFrame:
    df = download_fct_operator_hourly_summary(download_date)

    df2 = (
        df.groupby(["analysis_name", "month_first_day", "day_type", "departure_hour"])
        .agg({"n_trips": "sum"})
        .reset_index()
    )
    
    
    df2.columns = df2.columns.str.replace("_", " ").str.title()

    df2 = df2.rename(columns={"Month First Day": "Date"})

    df2['Date'] = df2['Date'].dt.strftime('%m-%Y')
    
    df2.to_parquet(
        f"{GTFS_DATA_DICT.gcs_paths.DIGEST_GCS}processed/fct_operator_hourly_summary_{download_date}.parquet"
    )
    return df2

In [22]:
clean_fct_operator_hourly_summary_df = clean_fct_operator_hourly_summary(date_str)


        SELECT 
            *
        FROM `cal-itp-data-infra-staging`.`tiffany_mart_gtfs_rollup`.`fct_operator_hourly_summary`
        WHERE month_first_day >=  DATE('2025-01-01')
    
Downloading: 100%|██████████|
download time: 0:00:07.547099


## Publish everything to GCS

In [23]:
GCS = GTFS_DATA_DICT["digest_tables_ah"].dir

ConfigKeyError: Missing key digest_tables_ah
    full_key: digest_tables_ah
    object_type=dict

In [ ]:
GTFS_DATA_DICT["digest_tables_ah"].schedule_rt_route_direction_summary

In [ ]:
file_names = [
    GTFS_DATA_DICT["digest_tables_ah"][f]
    for f in [
        "schedule_rt_route_direction_summary",
        "fct_monthly_operator_summary",
        "fct_operator_hourly_summary",
    ]
]

In [ ]:
file_names = [f"{key}_{date_str}.parquet" for key in file_names]

In [ ]:
file_names = [f"{GCS}{key}" for key in file_names]

In [ ]:
for f in file_names:
    publish_utils.write_to_public_gcs(f, f"gtfs_digest/{Path(f).name}", PUBLIC_GCS)

In [ ]:
for f in file_names:
    publish_public_data.export_parquet_as_csv_or_geojson(f, filetype="df")